Importamos las librerias que vamos a usar

In [30]:
import pandas as pd
import ast
import numpy as np

In [52]:
movies = pd.read_csv("../_src/Datasets/movies_dataset.csv")
credits = pd.read_csv("../_src/Datasets/credits.csv")

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_45256\954623710.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("./Dataset/movies_dataset.csv")


In [32]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [33]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Dataset credits

Notemos que segun el readme, lo unico que nos interesa del dataset de credits son los directores para la funcion get_director.


In [34]:
credits["crew"] = credits["crew"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
credits["crew"] = credits["crew"].apply(lambda x: ', '.join(genre['name'] for genre in x if genre["job"]=="Director") 
                                        if isinstance(x, list)  else np.nan)

In [35]:
credits.drop(columns="cast",inplace=True)

In [36]:
credits["crew"].value_counts()

                                                           887
John Ford                                                   63
Michael Curtiz                                              61
Alfred Hitchcock                                            52
Werner Herzog                                               52
                                                          ... 
Joung Yumi                                                   1
Melville W. Brown                                            1
Rob Cowan                                                    1
Yoshiharu Ashino, Vladimir Zhelezniakov, Aljosha Klimov      1
Daisy Asquith                                                1
Name: crew, Length: 18854, dtype: int64

In [37]:
credits["crew"].replace("",np.nan,inplace=True)

### Dataset movies

#### Eliminar las columnas que no serán utilizadas: `video, imdb_id, adult, original_title, poster_path y homepage`.


In [53]:
print(movies.shape)
movies.drop(columns=["video","imdb_id","adult","original_title","poster_path" ,"homepage"],inplace=True)
print(movies.shape)

(45466, 24)
(45466, 18)


#### Algunos campos, como `belongs_to_collection`, `production_companies` y otros (ver [Diccionario de datos](https://docs.google.com/spreadsheets/d/1QkHH5er-74Bpk122tJxy_0D49pJMIwKLurByOfmxzho/edit#gid=0)) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, vamos a desanidarlos para poder visualizar sus datos y unirlos al dataset para hacer alguna de las consultas de la API!

In [54]:
# Veo como son los tipos de cada columna anidada
columnas_anidadas = ["belongs_to_collection","genres","production_companies","production_countries","spoken_languages"]
for columna in columnas_anidadas:
    print(columna,": \n", movies[f"{columna}"][0])

belongs_to_collection : 
 {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
genres : 
 [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
production_companies : 
 [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries : 
 [{'iso_3166_1': 'US', 'name': 'United States of America'}]
spoken_languages : 
 [{'iso_639_1': 'en', 'name': 'English'}]


##### Notemos que en todos siempre aparece la key: "name" por lo que, solo me voy a quedar con esos datos

In [55]:
for columna in columnas_anidadas:
    movies[f"{columna}"] = movies[f"{columna}"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
    if columna == "belongs_to_collection":
        belongs_to_collection = pd.json_normalize(movies["belongs_to_collection"])
        movies["belongs_to_collection"] = belongs_to_collection["name"]
    else:
        movies[f"{columna}"] = movies[f"{columna}"].apply(lambda x: ', '.join(genre['name'] for genre in x) if isinstance(x, list) else np.nan)

In [56]:
for columna in columnas_anidadas:
    movies[f"{columna}"].replace("",np.nan,inplace=True)

#### Los valores nulos de los campos `revenue` y `budget` seran rellenados por el número 0.

In [57]:
print("revenue | budget")
print(movies["revenue"].isna().sum(),"\t" ,movies["budget"].isna().sum())
movies["revenue"].fillna(0,inplace=True)
movies["budget"].fillna(0,inplace=True)
print(movies["revenue"].isna().sum(),"\t",movies["budget"].isna().sum())

revenue | budget
6 	 0
0 	 0


#### Los valores nulos del campo `release date` deben eliminarse.

In [43]:
print(movies["release_date"].isnull().sum(),"nulos en el campo release_date")
movies.dropna(subset=["release_date"], inplace=True)
print(movies["release_date"].isnull().sum())

87 nulos en el campo release_date
0


#### De haber fechas, deberán tener el formato _AAAA-mm-dd_, además crearemos la columna `release_year` donde vamos a extraer el año de la fecha de estreno.

In [44]:
movies["release_date"] = pd.to_datetime(movies["release_date"],errors='coerce',format="%Y-%m-%d")
movies["release_year"] = movies["release_date"].dt.year

In [45]:
print(movies["release_date"].isnull().sum(),"nulos en el campo release_date")
movies.dropna(subset=["release_date"], inplace=True)
print(movies["release_date"].isnull().sum())

3 nulos en el campo release_date
0


#### Crearemos la columna con el retorno de inversión, llamada `return` con los campos _`revenue`_ y _`budget`_, dividiendo estas dos últimas, cuando no hay datos disponibles para calcularlo, tomaremos el valor 0.

In [46]:
movies["budget"] = pd.to_numeric(movies["budget"], errors='coerce')

movies["return"] = movies.apply(lambda row: row["revenue"] / row["budget"] if row["budget"] != 0 else 0, axis=1)

In [47]:
movies["return"].isna().sum()

0

#### Unimos los datasets en uno solo

In [48]:
movies["id"] = pd.to_numeric(movies["id"], errors='coerce',  downcast= "integer")
movies_clean = pd.merge(movies,credits,how="left",on="id")

Lo llevamos a csv

In [49]:
# movies_clean.to_csv("../_src/Datasets/movies_clean.csv",index=False)